In [1]:
import pandas as pd
import numpy as np


In [3]:
stockpd=pd.read_csv('tradebook-JC9287_conv.csv')
stockpd=stockpd.drop(['order_id','trade_id'], axis=1)

In [4]:
stockpd1=stockpd.groupby(["symbol","trade_type"])

In [5]:
stockpd1=stockpd1.sum()
print(stockpd1)

                       quantity    price
symbol     trade_type                   
ADANIENT   buy               49  3190.50
           sell               1   802.15
ADANIGAS   buy              228  3350.40
           sell              11   342.45
ADANIGREEN buy               23  1059.00
ADANIPORTS buy               39  2333.80
ADANIPOWER buy              349   310.60
ADANITRANS buy               12  1196.80
AFFLE      buy                2  5391.30
ALOKINDS   buy               80    51.65
APTECHT    buy               10   137.05
DISHTV     buy               20    15.95
DPSCLTD    buy               50    18.05
FCONSUMER  buy               30     8.40
FSL        buy              100   105.30
GMRINFRA   buy               25    26.75
GRAPHITE   buy               23   750.35
           sell               7   230.30
GRAVITY-X  buy              552    11.16
HCC        buy               30    10.75
HFCL       buy               10    51.20
IBREALEST  buy              300    60.90
IDEA       buy  

In [6]:
stockpd2 = stockpd1.reset_index()

In [7]:
stockpd2.loc[stockpd2['trade_type'] == 'sell', 'quantity'] = 0 - stockpd2.loc[stockpd2['trade_type'] == 'sell', 'quantity']

In [8]:
stockpd2=stockpd2.groupby('symbol').sum()

In [9]:
metadata=pd.read_csv('sec_bhavdata_full.csv')

,symbol,today_close_price
0,20MICRONS,58.55
1,21STCENMGM,54.35
2,3IINFOLTD,39.45
3,3MINDIA,25400.70
4,3PLAND,12.70
...,...,...
2081,ZOMATO,131.55
2082,ZOTA,367.90
2083,ZUARI,112.25
2084,ZUARIGLOB,133.00


In [10]:
# metadata.sort_values("SYMBOL", inplace=True)
# print(metadata)
metadata=metadata[["SYMBOL"," CLOSE_PRICE"]]
metadata.rename(columns={'SYMBOL': 'symbol', ' CLOSE_PRICE': 'today_close_price'}, inplace=True)

In [12]:
# metadata=metadata.set_index("symbol")
category_stock=pd.read_csv('NSE-Stock-LIST-1411-Stocks-Generated-on-25may2017.csv')
category_stock=category_stock[["Symbol","Sector"]]
category_stock.rename(columns={'Symbol': 'symbol'}, inplace=True)
print(category_stock)

metadata=metadata.join(category_stock.set_index("symbol"), how='left', on='symbol', lsuffix='x', rsuffix='y')
print(metadata)
stockpd2=stockpd2.join(metadata.set_index("symbol"), how='left', on='symbol')

          symbol       Sector
0      20MICRONS       Mining
1     3IINFOTECH    Computers
2        3MINDIA  Diversified
3        63MOONS          NaN
4        8KMILES    Computers
...          ...          ...
1406   ZODJRDMKJ    Jewellery
1407       ZUARI          NaN
1408   ZUARIGLOB          NaN
1409   ZYDUSWELL         Misc
1410       ZYLOG    Computers

[1411 rows x 2 columns]
          symbol  today_close_price       Sector
0      20MICRONS              58.55       Mining
1     21STCENMGM              54.35          NaN
2      3IINFOLTD              39.45          NaN
3        3MINDIA           25400.70  Diversified
4         3PLAND              12.70          NaN
...          ...                ...          ...
2081      ZOMATO             131.55          NaN
2082        ZOTA             367.90          NaN
2083       ZUARI             112.25          NaN
2084   ZUARIGLOB             133.00          NaN
2085   ZYDUSWELL            2054.50         Misc

[2086 rows x 3 columns]


In [13]:
stockpd2['final_valuation']=stockpd2['quantity']*stockpd2['today_close_price']
stockpd2=stockpd2.reset_index().groupby(["Sector"]).sum()
stockpd2

,quantity,price,today_close_price,final_valuation
Sector,,,,
Auto,3036,9912.25,483.70,1468513.20
Banks - PSU,350,90.85,42.05,7415.00
Banks - Private,80,10.40,9.35,748.00
Chemicals,25,1395.35,684.70,17117.50
Computers,110,242.35,538.75,23176.00
Construction,349,177.00,277.00,47120.85
Electronics,16,980.65,536.40,8582.40
Engineering,50,7.35,6.75,337.50
Entertainment,20,15.95,17.25,345.00


In [14]:
stockpd2=stockpd2.reset_index()
print(stockpd2)
print(len(stockpd2))

             Sector  quantity    price  today_close_price  final_valuation
0              Auto      3036  9912.25             483.70       1468513.20
1       Banks - PSU       350    90.85              42.05          7415.00
2   Banks - Private        80    10.40               9.35           748.00
3         Chemicals        25  1395.35             684.70         17117.50
4         Computers       110   242.35             538.75         23176.00
5      Construction       349   177.00             277.00         47120.85
6       Electronics        16   980.65             536.40          8582.40
7       Engineering        50     7.35               6.75           337.50
8     Entertainment        20    15.95              17.25           345.00
9           Finance       120    57.00            4085.25        122557.50
10   Infrastructure        25    26.75              40.55          1013.75
11             Misc        39  2333.80             693.00         27027.00
12  Pharmaceuticals      

In [18]:
from math import pi

import pandas as pd

from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum

from bokeh.plotting import figure, show, output_notebook

stockpd2['angle'] = stockpd2['final_valuation']/stockpd2['final_valuation'].sum() * 2*pi
stockpd2['color'] = Category20c[len(stockpd2)]
plot = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@Sector: @final_valuation")

plot.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Sector', source=stockpd2)

plot.axis.axis_label = None
plot.axis.visible = False
plot.grid.grid_line_color = None

# show(p)
# output_notebook()
show(plot)

In [167]:
output_notebook()
show(p)

RuntimeError: ignored